In [1]:
import pandas as pd

from utils import tag_mail

df = pd.read_json(r'data/gmail_turc_basic.txt', encoding = 'utf-8')

df['cat'] = df.headers.apply(tag_mail)

df.loc[df.cat=='amazon']

,snippet,date,headers,body,cat
941,"Bonjour TURC, Votre colis contenant NONZERS Ve...",2019-11-19 07:47:20,"[{'name': 'From', 'value': '""Amazon.fr"" <shipm...",Amazon.fr \n---------------------------------...,amazon
944,Amazon.fr Vos commandes | Votre compte | Amazo...,2019-11-19 01:38:38,"[{'name': 'From', 'value': '""Amazon.fr"" <confi...",Confirmation d'envoi Amazon.fr \nCommande #40...,amazon
955,Amazon.fr Vos commandes | Votre compte | Amazo...,2019-11-18 11:42:02,"[{'name': 'From', 'value': '""Amazon.fr"" <confi...",Confirmation de votre commande Amazon.fr \n \...,amazon
956,Merci d&#39;avoir visité Amazon.fr ! À votre d...,2019-11-18 11:38:39,"[{'name': 'From', 'value': 'Amazon <modificati...",Merci d'avoir visité Amazon.fr ! \n \n À vot...,amazon
957,Aide sur le mot de passe Pour vérifier votre i...,2019-11-18 11:38:04,"[{'name': 'From', 'value': '""Amazon.fr"" <accou...",Aide sur le mot de passe\n\nPour vérifier vo...,amazon
958,"Bonjour, Nous avons noté que vous avez récemme...",2019-11-18 11:37:15,"[{'name': 'From', 'value': 'Amazon <account-up...","Bonjour, \n \nNous avons noté que vous avez r...",amazon
1099,Offre à durée limitée Prime Video PrimeVideo.c...,2019-11-06 10:23:27,"[{'name': 'From', 'value': 'Amazon Prime Video...",======== \nAmazon Prime Video \n======== \n \...,amazon
1410,Offre à durée limitée Prime Video PrimeVideo.c...,2019-09-29 09:48:16,"[{'name': 'From', 'value': 'Amazon Prime Video...",======== \nAmazon Prime Video \n======== \n \...,amazon
1586,Offre à durée limitée Prime Video PrimeVideo.c...,2019-09-01 09:26:28,"[{'name': 'From', 'value': 'Amazon Prime Video...",======== \nAmazon Prime Video \n======== \n \...,amazon
1784,Offre spéciale Membres Prime : Téléchargez 2 l...,2019-08-01 10:56:50,"[{'name': 'From', 'value': '""Amazon.fr"" <annon...",\n \n \n \n \n==============================...,amazon


In [2]:
import re

import locale
locale.setlocale(locale.LC_TIME, "fr_FR")

from datetime import datetime

df_amazon = df.loc[df.cat=='amazon']

df_amazon = df_amazon.loc[df_amazon.body.str.contains("Votre Expédition ")]

res = []

for el in df.body:

    tmp = {}

    try:
        total_cost = re.findall(r'Montant total pour cet envoi : EUR \d{0,100000},\d\d', el)[0]
        tmp['total_cost'] = re.findall(r'\d{0,100000},\d\d',total_cost)[0]
    except:
        continue

    try:
        tmp['payment_tool'] = re.findall(r'Payé par (.*?): ', el)[0]
    except:#\n \n', el
        tmp['payment_tool'] = ''

    try:
        date = re.findall(r'Livraison : \n (.*?) \n \n', el, re.DOTALL)[0]
        date = datetime.strptime(date, '%A %d %B %Y')
    except:
        date = None

    tmp['date'] = date


    products = re.findall(r'\(Vendu par (.*?) \n \n', el, re.DOTALL)

    if products==[]:
        continue

    tmp['articles'] = []

    for prod in products:
        res_prod = {}
        res_prod['seller'] = prod.split(') :')[0]

        prod = prod.split('\n ')[1:]

        res_prod['article'] = prod[0]
        res_prod['price'] = re.findall(r'\d{0,100000},\d\d', prod[1])[0]

        tmp['articles'] += [res_prod]
    res+=[tmp]

    print(tmp)
    print('\n\n')
    print('#######################################################')
    print('\n\n')

res

{'total_cost': '7,29', 'payment_tool': 'Mastercard', 'date': datetime.datetime(2019, 11, 19, 0, 0), 'articles': [{'seller': 'George Official', 'article': "NONZERS Verre Trempé pour Huawei Honor 10, [3 Pack] 9H Dureté, HD Protecteur d'Écran en Verre Trempé, Anti-Rayures, sans Bulles, Anti-Trace de Doi ", 'price': '7,29'}]}



#######################################################



{'total_cost': '6,99', 'payment_tool': 'Mastercard', 'date': datetime.datetime(2019, 3, 28, 0, 0), 'articles': [{'seller': 'RAMPOW-FR', 'article': 'RampowT01 Câble USB Type C à USB 3.0 - Câble USB C Nylon Tressé en Fibre avec Connecteur Ultra Résistant - Chargeur USB C Charge/Synchro Ultime R ', 'price': '6,99'}]}



#######################################################



{'total_cost': '12,08', 'payment_tool': 'Mastercard', 'date': datetime.datetime(2019, 1, 11, 0, 0), 'articles': [{'seller': 'Yosen', 'article': 'POOPHUNS Verre Trempé Huawei Honor 10 [2 Pack] Protege Film Ecran Verre Trempé pour Huawei 

[{'total_cost': '7,29',
  'payment_tool': 'Mastercard',
  'date': datetime.datetime(2019, 11, 19, 0, 0),
  'articles': [{'seller': 'George Official',
    'article': "NONZERS Verre Trempé pour Huawei Honor 10, [3 Pack] 9H Dureté, HD Protecteur d'Écran en Verre Trempé, Anti-Rayures, sans Bulles, Anti-Trace de Doi ",
    'price': '7,29'}]},
 {'total_cost': '6,99',
  'payment_tool': 'Mastercard',
  'date': datetime.datetime(2019, 3, 28, 0, 0),
  'articles': [{'seller': 'RAMPOW-FR',
    'article': 'RampowT01 Câble USB Type C à USB 3.0 - Câble USB C Nylon Tressé en Fibre avec Connecteur Ultra Résistant - Chargeur USB C Charge/Synchro Ultime R ',
    'price': '6,99'}]},
 {'total_cost': '12,08',
  'payment_tool': 'Mastercard',
  'date': datetime.datetime(2019, 1, 11, 0, 0),
  'articles': [{'seller': 'Yosen',
    'article': 'POOPHUNS Verre Trempé Huawei Honor 10 [2 Pack] Protege Film Ecran Verre Trempé pour Huawei Honor 10 - Anti-Rayures - HD - Ultra Resistant Dureté 9H ',
    'price': '7,09'}]